In [14]:
%pip install orjson pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 314.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 488.9 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
import orjson
import os
import pandas as pd
from tqdm import tqdm
import joblib
from datetime import datetime as dt

In [2]:
tier = 'diamond'
output_dir = "/home/piddle/hdd/matches"
target_dir = os.path.join(output_dir, tier)
columns=['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT', 'result']

In [5]:
path = os.path.join(target_dir, os.listdir(target_dir)[6])
file = orjson.loads(open(path, 'rb').read())
file

[{'id': 'CNrwP4AK2XcfppRFUH1dlUptuzBJR7RwIqSCiwt2wkk=',
  'memo': None,
  'created_at': '2024-02-17T13:14:08+09:00',
  'game_map': 'SUMMONERS_RIFT',
  'queue_info': {'id': 420,
   'queue_translate': '개인/2인 랭크 게임',
   'game_type': 'SOLORANKED'},
  'version': '14.3.558.8314',
  'game_length_second': 915,
  'is_remake': False,
  'is_opscore_active': True,
  'is_recorded': False,
  'average_tier_info': {'tier': 'DIAMOND',
   'division': 3,
   'tier_image_url': 'https://opgg-static.akamaized.net/images/medals_new/diamond.png',
   'border_image_url': 'https://opgg-static.akamaized.net/images/border_new/diamond.png'},
  'participants': [{'summoner': {'id': 7664181,
     'summoner_id': 'gl_ZESBhCNsXKPjXsQvX78denPtyp2Vzg1kbevUKDbNpQg',
     'acct_id': 'jOa-tchpHEzPHJ7bXW0-QOADa6TNUJIt-r1zPmGeQeyK',
     'puuid': 'YPLElOWunu_dVHurGGAURTPQZmDFoTaIamvJqY2_RJq3WX5EDYnzw1OsGVoGoYGT1MTP-7Eh3dw27w',
     'game_name': 'eun seob',
     'tagline': 'KR1',
     'name': 'eun seob',
     'internal_name': 'eu

In [6]:
participants = pd.DataFrame(columns=columns)

for match in file:
    win = {'result': 1}
    lose = {'result': 0}

    for participant in match['participants']:
        if participant['stats']['result'] == 'WIN':
            win[participant['position']] = participant['champion_id']
        else:
            lose[participant['position']] = participant['champion_id']

    participants = pd.concat([participants, pd.DataFrame([win]), pd.DataFrame([lose])], ignore_index=True)

participants

,TOP,JUNGLE,MID,ADC,SUPPORT,result
0,77,76,777,901,57,1
1,164,876,84,18,902,0
2,164,141,517,21,53,1
3,134,63,126,901,57,0
4,164,876,166,236,902,1
5,8,141,777,222,40,0
6,517,64,23,202,26,1
7,910,104,54,498,497,0
8,92,76,711,360,412,1
9,164,5,84,236,78,0


In [13]:
def parse_match(file_iter) -> pd.DataFrame:
    from tqdm import tqdm
    import orjson


    for file in tqdm(file_iter):
        try:
            json_data = orjson.loads(open(file.path, 'rb').read()) 

        except:
            print('error: ', file.path)
            continue

        for match in json_data:
            win = {'result': 1}
            lose = {'result': 0}

            for participant in match['participants']:
                if participant['stats']['result'] == 'WIN':
                    win[participant['position']] = participant['champion_id']
                elif participant['stats']['result'] == 'LOSE':
                    lose[participant['position']] = participant['champion_id']
                else:
                    break
            else:
                yield pd.DataFrame([win, lose], columns=columns)


In [14]:
file_iter = os.scandir(target_dir)
participants = pd.DataFrame(columns=columns)
participants.to_csv(f'../data/{tier}_matches.csv', mode='w', index=False)
    
for df in parse_match(file_iter):
    df.to_csv(f'../data/{tier}_matches.csv', mode='a', index=False, header=False)

63664it [23:12, 45.72it/s] 


In [3]:
df = pd.read_csv(f'../data/{tier}_matches.csv')
df.dropna(inplace=True)
df = df.astype(int)
df.to_csv(f'../data/{tier}_matches.csv', mode='w', index=False)

In [8]:
df = pd.read_csv(f'../data/{tier}_matches.csv')
for idx, pos in zip(range(10000, 50001, 10000), columns):
    df[pos] = df[pos] + idx

df

,TOP,JUNGLE,MID,ADC,SUPPORT,result
0,10078,20064,30055,40029,50040,1
1,10200,20234,30034,40221,50043,0
2,10041,20011,30055,40081,50518,1
3,10122,20024,30091,40110,50057,0
4,10266,20024,30055,40429,50043,1
...,...,...,...,...,...,...
1260637,10164,20028,30268,40236,50080,0
1260638,10910,20064,30266,40236,50902,1
1260639,10014,20063,30163,40360,50111,0
1260640,10055,20234,30107,40236,50497,1


In [9]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

import itertools

In [10]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.combinations = [list(element) for element in itertools.combinations(range(5), 2)]
        self.combinations_len = len(self.combinations)

    def __getitem__(self, idx):
        df_idx = idx // self.combinations_len
        combination_idx = idx % self.combinations_len

        x = self.df.iloc[df_idx, self.combinations[combination_idx]]
        x = torch.tensor(x, dtype=torch.int32, device='cuda')

        y = self.df.iloc[df_idx, -1]
        y = torch.tensor(y, dtype=torch.float64, device='cuda')

        return x, y
        
    def __len__(self):
        return len(self.combinations) * len(self.df)

In [11]:
class SimilarityModel(nn.Module):
    def __init__(self, config):
        super(SimilarityModel, self).__init__()
        self.embedding = nn.Embedding(config['n_layers'], config['emb_size'])


    def forward(self, input):
        input = input.transpose(0, 1)
        embedded = self.embedding(input)
        A = embedded[0]
        B = embedded[1]
        output = F.cosine_similarity(A, B, dim=1)
        output = torch.abs(output)

        return output

In [16]:
df = pd.read_csv(f'../data/{tier}_matches.csv')

config = {}
config['n_layers'] = int(df.drop(columns='result').max().max() + 1)
config['emb_size'] = 128

dataset = MyDataset(df)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = SimilarityModel(config).to('cuda')

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

model.train()
for x, y in tqdm(loader):
    optimizer.zero_grad()
    output = model(x)

    # print(output, y)
    loss = loss_fun(output, y)
    loss.backward()
    optimizer.step()

  0%|          | 0/2521284 [00:00<?, ?it/s]/tmp/ipykernel_842208/378348694.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = torch.tensor(x, dtype=torch.int32, device='cuda')
100%|██████████| 2521284/2521284 [2:39:29<00:00, 263.48it/s]  


In [23]:
now = dt.now().strftime('%Y-%m-%d_%H-%M')
joblib.dump(model, f'../data/cossim-{now}.joblib')

['../data/cossim-2024-03-07_12-15.joblib']